# Test LLM calling
- test different ways of calling LLMS, native API, LangChain, sync/async


In [1]:
# to selectively re-import as needed
import sys
# del sys.modules['ainb_llm']
# del sys.modules['ainb_const']
# del sys.modules['ainb_utilities']
# del sys.modules['ainb_webscrape']
# del sys.modules['AInewsbot_langgraph']



In [2]:
import os
import shutil
# import dotenv
# import subprocess

from collections import Counter
import json
import uuid
import re
# import operator
import pickle
from datetime import datetime, timedelta

import langchain
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_anthropic import ChatAnthropic

from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import (ChatPromptTemplate, PromptTemplate,
                                    SystemMessagePromptTemplate, HumanMessagePromptTemplate)
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser

from langgraph.checkpoint.memory import MemorySaver
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.errors import NodeInterrupt
from langchain.globals import set_debug

import numpy as np
import pandas as pd
import umap

import sklearn

import bs4

from tenacity import (
    retry,
    stop_after_attempt,
    wait_exponential,
    retry_if_exception_type
)

import asyncio
from asyncio import Semaphore

from IPython.display import HTML, Image, Markdown, display

# import pyperclip
# import shlex

import openai
from openai import OpenAI

from google.cloud import aiplatform
from vertexai.language_models import TextGenerationModel
import google.generativeai as genai

import anthropic
from anthropic import Anthropic

from pydantic import BaseModel, Field
from typing import List, TypedDict, Annotated, Any

import httpx

import trafilatura   # web scrape uses this to get clean news stories w/o a lot of js and boilerplate

from ainb_const import (
                        
                        REWRITE_PROMPT, FINAL_SUMMARY_PROMPT,
                        SCREENSHOT_DIR, SUMMARIZE_SYSTEM_PROMPT, SUMMARIZE_USER_PROMPT
                       )

from ainb_utilities import log

from AInewsbot_langgraph import (newscatcher_sources, fn_initialize, fn_download_sources, fn_extract_urls,
                                 fn_verify_download, fn_extract_newscatcher, fn_filter_urls, fn_topic_clusters,
                                 fn_topic_analysis, fn_download_pages, fn_summarize_pages, fn_propose_cats,
                                 fn_compose_summary, fn_rewrite_summary, fn_is_revision_complete, fn_send_mail
                                )


import podcastfy
from podcastfy.client import generate_podcast, process_content

from selenium.webdriver.support.ui import WebDriverWait
from IPython.display import Audio, display, Markdown

import pdb

# need this to run async in jupyter since it already has an asyncio event loop running
import asyncio
import nest_asyncio
nest_asyncio.apply()

# Activate global verbose logging
set_debug(False)


In [3]:
print(f"Python            {sys.version}")
print(f"LangChain         {langchain.__version__}")
print(f"OpenAI            {openai.__version__}")
print(f"Anthropic         {anthropic.__version__}")
# print(f"smtplib           {smtplib.sys.version}")
print(f"trafilatura       {trafilatura.__version__}")
# print(f"bs4               {bs4.__version__}")
print(f"numpy             {np.__version__}")
print(f"pandas            {pd.__version__}")
print(f"sklearn           {sklearn.__version__}")
print(f"umap              {umap.__version__}")
print(f"podcastfy         {podcastfy.__version__}")


Python            3.11.11 | packaged by conda-forge | (main, Dec  5 2024, 14:21:42) [Clang 18.1.8 ]
LangChain         0.3.18
OpenAI            1.63.1
Anthropic         0.45.2
trafilatura       2.0.0
numpy             1.26.4
pandas            2.2.3
sklearn           1.6.1
umap              0.5.7
podcastfy         0.4.1


In [4]:
#  LOWCOST_MODEL, MODEL, HIGHCOST_MODEL = 'gpt-4o-mini', 'gpt-4o-2024-11-20', 'o3-mini'

In [5]:
# a basic LLM call with langchain\
openai_model = ChatOpenAI(model="gpt-4o-mini")

response = openai_model.invoke([
    SystemMessage(content="You are a translator. Translate the following from English into Italian"),
    HumanMessage(content='Listen to me. You are beautiful. You are perfect and I love you.'),
])

response

2025-02-25 10:56:00,505 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content='Ascoltami. Sei bellissima. Sei perfetta e ti amo.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 39, 'total_tokens': 56, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7fcd609668', 'finish_reason': 'stop', 'logprobs': None}, id='run-1c2a9760-8483-46fa-9c04-b0cb530255fc-0', usage_metadata={'input_tokens': 39, 'output_tokens': 17, 'total_tokens': 56, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
response.response_metadata

{'token_usage': {'completion_tokens': 17,
  'prompt_tokens': 39,
  'total_tokens': 56,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_name': 'gpt-4o-mini-2024-07-18',
 'system_fingerprint': 'fp_7fcd609668',
 'finish_reason': 'stop',
 'logprobs': None}

In [7]:
response.usage_metadata
# no rate limit info like tokens remaining, available in headers

{'input_tokens': 39,
 'output_tokens': 17,
 'total_tokens': 56,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [8]:
client = openai.OpenAI()

# Retrieve the list of available models
models = client.models.list()

# Print out the model IDs
models = [model.id for model in models.data]
models.sort()
print("\n".join(models))
# yay, we got o3-mini API access

2025-02-25 10:56:03,760 - httpx - INFO - HTTP Request: GET https://api.openai.com/v1/models "HTTP/1.1 200 OK"


babbage-002
chatgpt-4o-latest
dall-e-2
dall-e-3
davinci-002
gpt-3.5-turbo
gpt-3.5-turbo-0125
gpt-3.5-turbo-1106
gpt-3.5-turbo-16k
gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
gpt-4
gpt-4-0125-preview
gpt-4-0613
gpt-4-1106-preview
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4-turbo-preview
gpt-4o
gpt-4o-2024-05-13
gpt-4o-2024-08-06
gpt-4o-2024-11-20
gpt-4o-audio-preview
gpt-4o-audio-preview-2024-10-01
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini
gpt-4o-mini-2024-07-18
gpt-4o-mini-audio-preview
gpt-4o-mini-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-realtime-preview
gpt-4o-realtime-preview-2024-10-01
gpt-4o-realtime-preview-2024-12-17
o1
o1-2024-12-17
o1-mini
o1-mini-2024-09-12
o1-preview
o1-preview-2024-09-12
o3-mini
o3-mini-2025-01-31
omni-moderation-2024-09-26
omni-moderation-latest
text-embedding-3-large
text-embedding-3-small
text-embedding-ada-002
tts-1
tts-1-1106
tts-1-hd
tts-1-hd-1106
whisper-1


In [9]:
def list_gemini_models():
    try:
        # Configure the library
        genai.configure()

        # List available models
        models = genai.list_models()

        print("Available Gemini Models:")
        print("-----------------------")
        for m in models:
            if "gemini" in m.name.lower():
                print(f"Name: {m.name}")
                print(f"Description: {m.description}")
                print(f"Generation Methods: {m.supported_generation_methods}")
                print("-----------------------")

    except Exception as e:
        print(f"An error occurred: {str(e)}")

list_gemini_models()


Available Gemini Models:
-----------------------
Name: models/gemini-1.0-pro-latest
Description: The original Gemini 1.0 Pro model. This model will be discontinued on February 15th, 2025. Move to a newer Gemini version.
Generation Methods: ['generateContent', 'countTokens']
-----------------------
Name: models/gemini-1.0-pro
Description: The best model for scaling across a wide range of tasks
Generation Methods: ['generateContent', 'countTokens']
-----------------------
Name: models/gemini-pro
Description: The best model for scaling across a wide range of tasks
Generation Methods: ['generateContent', 'countTokens']
-----------------------
Name: models/gemini-1.0-pro-001
Description: The original Gemini 1.0 Pro model version that supports tuning. Gemini 1.0 Pro will be discontinued on February 15th, 2025. Move to a newer Gemini version.
Generation Methods: ['generateContent', 'countTokens', 'createTunedModel']
-----------------------
Name: models/gemini-1.0-pro-vision-latest
Description

In [10]:
# try Gemini
# GEMINI_MODEL = "gemini-1.5-pro"  # or "flash" depending on the desired model
GEMINI_MODEL = "gemini-2.0-flash-exp"

gmodel = ChatGoogleGenerativeAI(
    model=GEMINI_MODEL,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

messages = [
    SystemMessage(content="You are a translator. Translate the following from English into Italian without explanation or comment."),
    HumanMessage(content="Listen to me. You are beautiful. You are perfect and I love you."),
]

# Invoke the model
response = gmodel.invoke(messages)

# Print the response
print(response.content)

Ascoltami. Sei bellissima. Sei perfetto/a e ti amo.



In [11]:
# Initialize the Anthropic client with your API key
client = Anthropic(api_key=os.environ.get("CLAUDE_API_KEY"))

# List available models
models = client.models.list()

# Print model information
print("Available Anthropic Models:")
for model in models.data:
    print(f"- {model.id}")
    print(f"  Description: {model.display_name}")
    print(f"  Created: {model.created_at}")
    print()

2025-02-25 10:56:09,271 - httpx - INFO - HTTP Request: GET https://api.anthropic.com/v1/models "HTTP/1.1 200 OK"


Available Anthropic Models:
- claude-3-7-sonnet-20250219
  Description: Claude 3.7 Sonnet
  Created: 2025-02-19 00:00:00+00:00

- claude-3-5-sonnet-20241022
  Description: Claude 3.5 Sonnet (New)
  Created: 2024-10-22 00:00:00+00:00

- claude-3-5-haiku-20241022
  Description: Claude 3.5 Haiku
  Created: 2024-10-22 00:00:00+00:00

- claude-3-5-sonnet-20240620
  Description: Claude 3.5 Sonnet (Old)
  Created: 2024-06-20 00:00:00+00:00

- claude-3-haiku-20240307
  Description: Claude 3 Haiku
  Created: 2024-03-07 00:00:00+00:00

- claude-3-opus-20240229
  Description: Claude 3 Opus
  Created: 2024-02-29 00:00:00+00:00

- claude-3-sonnet-20240229
  Description: Claude 3 Sonnet
  Created: 2024-02-29 00:00:00+00:00

- claude-2.1
  Description: Claude 2.1
  Created: 2023-11-21 00:00:00+00:00

- claude-2.0
  Description: Claude 2.0
  Created: 2023-07-11 00:00:00+00:00



In [12]:
# anthropic

# Initialize the ChatAnthropic model
claude_model = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    anthropic_api_key=os.environ["CLAUDE_API_KEY"],
    temperature=0,
)

# Create the messages
messages = [
    SystemMessage(content="You are a translator. Translate the following from English into Italian without explanation or comment."),
    HumanMessage(content="Listen to me. You are beautiful. You are perfect and I love you."),
]

# Invoke the model
response = claude_model.invoke(messages)

# Print the response
print(response.content)

2025-02-25 10:56:10,706 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


Ascoltami. Sei bellissima. Sei perfetta e ti amo.


In [13]:
# use a langchain template
system_template = "You are a translator. Translate the following from English into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
parser = StrOutputParser()
chain = prompt_template | openai_model | parser
chain.invoke({"language": "italian", "text": "hi"})


2025-02-25 10:56:11,584 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


'Ciao! Come posso aiutarti oggi?'

In [14]:
# time multiple templates (single-threaded)
prompt_inputs = [
    {"language": "French", "adjective1": "flawless", "adjective2": "beautiful"},
    {"language": "German", "adjective1": "immaculate", "adjective2": "exquisite"},
    {"language": "Spanish", "adjective1": "perfect", "adjective2": "gorgeous"},
    {"language": "Italian", "adjective1": "amazing", "adjective2": "magnificent"},
    {"language": "Hungarian", "adjective1": "ravishing", "adjective2": "stunning"},
]
system_template = 'You are a translator. Translate the following text from English into {language}. Provide only the translation, no other information:'
user_template = 'Listen to me. You are {adjective1}. You are {adjective2} and I love you.'

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template),
     ("user", user_template)]
)

parser = StrOutputParser()

ochain = prompt_template | openai_model | parser
gchain = prompt_template | gmodel | parser
cchain = prompt_template | claude_model | parser

start_time = datetime.now()
for tpl in prompt_inputs:
    for chain in [ochain, gchain, cchain]:
        response = ""
        #     print()
        #     print(prompt_template.format(**tpl))
        # stream tokens as they are generated
        for r in chain.stream(tpl):
            print(r, end="")
            response += r
        print()
end_time = datetime.now()

difference = end_time - start_time
total_seconds = difference.total_seconds()
print(f"\n\nElapsed seconds: {total_seconds:.6f}")


2025-02-25 10:56:12,929 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Écoute-moi. Tu es parfaite. Tu es belle et je t'aime.
Écoute-moi. Tu es parfait(e). Tu es magnifique et je t'aime.



2025-02-25 10:56:14,057 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


Écoute-moi. Tu es parfait(e). Tu es magnifique et je t'aime.


2025-02-25 10:56:14,728 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

Ascoltami. Sei fantastico/a. Sei magnifico/a e ti amo.


2025-02-25 10:56:27,381 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Figyelj rám. Elbűvölő vagy. Lélegzetelállító vagy és szeretlek.
Hallgass rám. Elragadó vagy. Lenyűgöző vagy, és szeretlek.



2025-02-25 10:56:29,058 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"


Figyelj rám. Elragadó vagy. Gyönyörű vagy és szeretlek.


Elapsed seconds: 17.386070


In [15]:
# Retry decorator with exponential backoff
# if I run google too fast I get a 429 error, need to update something in gcp probably

def should_retry_exception(exception):
    """Determine if the exception should trigger a retry. (always retry)"""
    print(type(exception))
    print(exception)
    return True


# @retry(
#     stop=stop_after_attempt(8),  # Maximum 8 attempts
#     wait=wait_exponential(multiplier=1, min=2, max=128),  # Wait 2^x * multiplier seconds between retries
#     retry=retry_if_exception_type(should_retry_exception),
#     before_sleep=lambda retry_state: print(f"Retrying after {retry_state.outcome.exception()}, attempt {retry_state.attempt_number}")
# )
async def process_translation(chain, inputs, name):
    response = ""
    async for chunk in chain.astream(inputs):
        response += chunk
    return response, name


async def main():
    prompt_inputs = [
        {"language": "French", "adjective1": "flawless", "adjective2": "beautiful"},
        {"language": "German", "adjective1": "immaculate", "adjective2": "exquisite"},
        {"language": "Spanish", "adjective1": "perfect", "adjective2": "gorgeous"},
        {"language": "Italian", "adjective1": "amazing", "adjective2": "magnificent"},
        {"language": "Hungarian", "adjective1": "ravishing", "adjective2": "stunning"},
    ]

    system_template = 'You are a translator. Translate the following text into {language}. Provide only the translation, no other information:'
    user_template = 'Listen to me. You are {adjective1}. You are {adjective2} and I love you.'

    prompt_template = ChatPromptTemplate.from_messages([
        ("system", system_template),
        ("user", user_template)
    ])

    parser = StrOutputParser()
    ochain = prompt_template | openai_model | parser
    gchain = prompt_template | gmodel | parser
    cchain = prompt_template | claude_model | parser
    chains = {'openai': ochain, 'google': gchain, 'claude': cchain}

    start_time = datetime.now()

    tasks = []
    for tpl in prompt_inputs:
        print(f"Queuing {tpl['language']} translations...")
        for name, chain in chains.items():
            task = asyncio.create_task(process_translation(chain, tpl, name))
            tasks.append(task)

    try:
        responses = await asyncio.gather(*tasks)
        for response, name in responses:
            print(f"{name}: {response}\n")
    except Exception as e:
        print(f"Error during translation: {str(e)}")

    end_time = datetime.now()
    difference = end_time - start_time
    total_seconds = difference.total_seconds()
    print(f"\nElapsed seconds: {total_seconds:.6f}")

asyncio.run(main())


Queuing French translations...
Queuing German translations...
Queuing Spanish translations...
Queuing Italian translations...
Queuing Hungarian translations...


2025-02-25 10:56:29,598 - langchain_google_genai.chat_models - WARNING - Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
2025-02-25 10:56:29,779 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-25 10:56:29,867 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-02-25 10:56:29,874 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-25 10:56:29,877 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-25 10:56:29,912 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-02-25 10:56:29,925 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-25 10:56:29,94

openai: Écoute-moi. Tu es parfaite. Tu es belle et je t'aime.

google: Écoute-moi. Tu es parfait(e). Tu es magnifique et je t'aime.


claude: Écoute-moi. Tu es parfait(e). Tu es magnifique et je t'aime.

openai: Hör mir zu. Du bist makellos. Du bist exquisit und ich liebe dich.

google: Hör mir zu. Du bist makellos. Du bist exquisit und ich liebe dich.


claude: Hör mir zu. Du bist makellos. Du bist wunderschön und ich liebe dich.

openai: Escúchame. Eres perfecto. Eres hermosa y te amo.

google: Escúchame. Eres perfecto/a. Eres precioso/a y te amo.


claude: Escúchame. Eres perfecta. Eres preciosa y te amo.

openai: Ascoltami. Sei fantastico. Sei magnifico e ti amo.

google: Ascoltami. Sei fantastico/a. Sei magnifico/a e ti amo.


claude: Ascoltami. Sei fantastico. Sei magnifico e ti amo.

openai: Figyelj rám. Elbűvölő vagy. Lélegzetelállító vagy, és szeretlek.

google: Hallgass rám. Elbűvölő vagy. Lenyűgöző vagy, és szeretlek.


claude: Figyelj rám. Elragadó vagy. Gyönyörű vagy és sz

In [16]:
# same but use ainvoke, no stream
# Rate limit settings
# CALLS_PER_MINUTE = 60  # Adjust based on your quota
# MAX_CONCURRENT = 5     # Maximum concurrent requests
# sem = Semaphore(MAX_CONCURRENT) # semaphore for controlling concurrent requests

# @sleep_and_retry
# @limits(calls=CALLS_PER_MINUTE, period=60)
@retry(
    stop=stop_after_attempt(8),  # Maximum 8 attempts
    wait=wait_exponential(multiplier=1, min=2, max=128),  # Wait 2^x * multiplier seconds between retries
    retry=retry_if_exception_type(should_retry_exception),
    before_sleep=lambda retry_state: print(f"Retrying after {retry_state.outcome.exception()}, attempt {retry_state.attempt_number}")
)
async def async_langchain(chain, input_dict, name=""):
#     async with sem:
        response = await chain.ainvoke(input_dict)
        return response, name

prompt_templates = [
    {"language": "French", "adjective1": "flawless", "adjective2": "beautiful"},
    {"language": "German", "adjective1": "immaculate", "adjective2": "exquisite"},
    {"language": "Spanish", "adjective1": "perfect", "adjective2": "gorgeous"},
    {"language": "Italian", "adjective1": "amazing", "adjective2": "magnificent"},
    {"language": "Hungarian", "adjective1": "ravishing", "adjective2": "stunning"},
]

chains = {'openai': ochain, 
          'google': gchain, 
          'claude': cchain}

start_time = datetime.now()
tasks = []
for d in prompt_templates:
    for name, chain in chains.items():
        task = asyncio.create_task(async_langchain(chain, d, name))
        tasks.append(task)
responses = await asyncio.gather(*tasks)
end_time = datetime.now()

difference = end_time - start_time
total_seconds = difference.total_seconds()
for response, name in responses:
    print(f"{name}: {response}")
print(f"\n\nElapsed seconds: {total_seconds:.6f}")


2025-02-25 10:56:35,020 - langchain_google_genai.chat_models - WARNING - Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
2025-02-25 10:56:35,021 - langchain_google_genai.chat_models - WARNING - Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..
2025-02-25 10:56:35,455 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-25 10:56:35,603 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-25 10:56:35,613 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
2025-02-25 10:56:35,677 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages "HTTP/1.1 200 OK"
20

openai: Écoute-moi. Tu es parfaite. Tu es belle et je t'aime.
google: Écoute-moi. Tu es parfait(e). Tu es magnifique et je t'aime.

claude: Écoute-moi. Tu es parfait(e). Tu es magnifique et je t'aime.
openai: Hör mir zu. Du bist makellos. Du bist exquisit und ich liebe dich.
google: Hör mir zu. Du bist makellos. Du bist exquisit und ich liebe dich.

claude: Hör mir zu. Du bist makellos. Du bist wunderschön und ich liebe dich.
openai: Escúchame. Eres perfecto. Eres hermosa y te amo.
google: Escúchame. Eres perfecto/perfecta. Eres precioso/preciosa y te amo/quiero.

claude: Escúchame. Eres perfecta. Eres preciosa y te amo.
openai: Ascoltami. Sei straordinario. Sei magnifico e ti amo.
google: Ascoltami. Sei fantastico/a. Sei magnifico/a e ti amo.

claude: Ascoltami. Sei fantastico/a. Sei magnifico/a e ti amo.
openai: Hallgass rám. Elbűvölő vagy. Lenyűgöző vagy, és szeretlek.
google: Hallgass rám. Elbűvölő vagy. Lenyűgöző vagy, és szeretlek.

claude: Figyelj rám. Elragadó vagy. Gyönyörű va

In [17]:
# test o3-mini
client = OpenAI()
response = client.chat.completions.create(
    model="o3-mini",
    reasoning_effort = "low",
    messages=[
        {
            "role": "system",
            "content": "You will act as an expert Python developer."
        },
        {
            "role": "user",
            "content": "Write a Python script that takes a matrix represented as a string with format '[1,2],[3,4],[5,6]' and prints the transpose in the same format."
        }
    ]
)
print(response.choices[0].message.content)


2025-02-25 10:56:53,330 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Below is one way to write a Python script that accomplishes this task:

---------------------------

#!/usr/bin/env python3
import sys

def parse_matrix(matrix_str):
    # First, remove spaces and then split on '],'
    # Remove leading and trailing square brackets if present
    matrix_str = matrix_str.strip()
    # Remove any outer brackets - if the string was something like '[[1,2],[3,4]]'
    if matrix_str.startswith('[') and matrix_str.endswith(']'):
        matrix_str = matrix_str[1:-1]
    # Now split along '],'
    rows = []
    # A simple approach: split by '],'
    parts = matrix_str.split('],')
    for part in parts:
        # cleanup part
        part = part.strip('[] ')
        if part:
            # convert each number to an integer
            numbers = [int(x.strip()) for x in part.split(',')]
            rows.append(numbers)
    return rows

def transpose(matrix):
    # Use zip(*matrix) to produce the transpose.
    return list(map(list, zip(*matrix)))

def format_matr

In [18]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", "You will act as an expert Python developer."),
     ("user", "{input}")]
)
parser = StrOutputParser()
openai_model = ChatOpenAI(model="o3-mini", reasoning_effort="low")
ochain = prompt_template | openai_model | parser
response = ochain.invoke("Write a Python script that takes a matrix represented as a string with format '[1,2],[3,4],[5,6]' and prints the transpose in the same format.")

print(response)


2025-02-25 10:57:13,712 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Below is one solution that reads the matrix from a string, computes its transpose, and prints the result in the same format:

------------------------------------------------------------
#!/usr/bin/env python3
import sys

def parse_matrix(matrix_str):
    """
    Given a string like "[1,2],[3,4],[5,6]",
    convert it to a list of lists [[1, 2], [3, 4], [5, 6]].
    """
    # Surround the string with brackets to create a valid literal
    try:
        # Use eval safely by controlling globals and locals
        matrix = eval(f'[{matrix_str}]', {"__builtins__":None}, {})
    except Exception as e:
        print("Error parsing matrix:", e)
        sys.exit(1)
    return matrix

def transpose(matrix):
    """
    Transpose the given matrix.
    """
    if not matrix:
        return []
    # Use zip(*matrix) to extract the columns as rows
    return [list(row) for row in zip(*matrix)]

def format_matrix(matrix):
    """
    Format the matrix as a string in the form "[a,b],[c,d],..."
    """

In [19]:
# could use the metadata to saturate the OpenAI API and use as many tokens per second as available
# but not supported by langchain across multiple models so exponential backoff seems to be the best alternative

apikey = os.environ.get("OPENAI_API_KEY")
url = "https://api.openai.com/v1/chat/completions"
headers = {
    "OpenAI-Beta": "assistants=v2",
    "Authorization": f"Bearer {apikey}"
}
body = {
    "model": "gpt-4o-2024-08-06", "max_tokens": 25, "top_p": 0.8,
    "messages": [
        {"role": "user",
         "content": [{"type": "text", "text": "What is the airspeed velocity of an unladen swallow"}]
        }
    ]}

try:
    response = httpx.post(url, headers=headers, json=body)
    response_json = response.json()
    print(json.dumps(response_json, indent=3))

    # Extract headers starting with 'x-' and load them into a dictionary
    x_headers = {k: v for k, v in response.headers.items() if k.lower().startswith('x-')}

    # Convert time values into seconds
    time_multipliers = {'h': 3600, 'm': 60, 's': 1, 'ms': 0.001}
    rate_headers = ['x-ratelimit-limit-requests', 'x-ratelimit-limit-tokens',
                    'x-ratelimit-remaining-requests', 'x-ratelimit-remaining-tokens',
                    'x-ratelimit-reset-requests', 'x-ratelimit-reset-tokens']
    for key in rate_headers:
        if key in x_headers:
            if 'reset' in key:
                total_time = 0
                for time_part in re.findall(r'(\d+)([hms]+)', x_headers[key]):
                    total_time += int(time_part[0]) * time_multipliers[time_part[1]]
                x_headers[key] = total_time
            else:
                x_headers[key] = int(x_headers[key])

    # Print the headers
    print("\nHeaders starting with 'x-':")
    for key, value in x_headers.items():
        print(f"{key}: {value}")

except Exception as e:
    print(e)
    raise


2025-02-25 10:57:14,661 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{
   "id": "chatcmpl-B4razvoi1wanjloNgST3PfijerAsr",
   "object": "chat.completion",
   "created": 1740499033,
   "model": "gpt-4o-2024-08-06",
   "choices": [
      {
         "index": 0,
         "message": {
            "role": "assistant",
            "content": "The question about the airspeed velocity of an unladen swallow is a humorous reference from the movie \"Monty Python and the",
            "refusal": null
         },
         "logprobs": null,
         "finish_reason": "length"
      }
   ],
   "usage": {
      "prompt_tokens": 18,
      "completion_tokens": 25,
      "total_tokens": 43,
      "prompt_tokens_details": {
         "cached_tokens": 0,
         "audio_tokens": 0
      },
      "completion_tokens_details": {
         "reasoning_tokens": 0,
         "audio_tokens": 0,
         "accepted_prediction_tokens": 0,
         "rejected_prediction_tokens": 0
      }
   },
   "service_tier": "default",
   "system_fingerprint": "fp_eb9dce56a8"
}

Headers starting with 'x-

In [20]:
# structured response from openai using pydantic and openai api
client = OpenAI()

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "Extract the event information in JSON format."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed

print(json.dumps(json.loads(event.json()), indent=2))

event

2025-02-25 10:57:17,871 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{
  "name": "Science Fair",
  "date": "Friday",
  "participants": [
    "Alice",
    "Bob"
  ]
}


CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])

In [21]:
# using langchain and with_structured_output
formatted_date = datetime.now().strftime('%Y-%m-%d')

system_prompt = f"""You are a precise calendar event extractor. Your task is to extract event details from natural language and format them as structured data.

TASK REQUIREMENTS:
1. Extract exactly three pieces of information:
   - Event name (be specific and descriptive)
   - Event date (convert relative dates to YYYY-MM-DD format)
   - List of all participants mentioned

RULES:
- Convert relative dates using today's date ({formatted_date}) as reference
- Include a descriptive event name even if only implied
- List ALL participants mentioned, even in passing
- Never include participants who aren't explicitly mentioned
- If any required information is missing, make reasonable assumptions based on context

Example Input: "Alice and Bob are going to a science fair on Friday"

Example Output:
{{{{
    "name": "Science Fair",
    "date": "2024-02-02",
    "participants": ["Alice", "Bob"]
}}}}

FORMAT INSTRUCTIONS:
The output should be a JSON object with the following schema:
{{{{
    "name": string,       // The name or title of the event
    "date": string,       // The date in YYYY-MM-DD format
    "participants": [     // Array of participant names
        string,
        ...
    ]
}}}}
"""

class CalendarEvent(BaseModel):
    name: str = Field(description="The name or title of the event")
    date: str = Field(description="The date of the event in ISO format (YYYY-MM-DD)")
    participants: List[str] = Field(description="List of people participating in the event")

parser = PydanticOutputParser(pydantic_object=CalendarEvent)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{input_text}")
])

# Create the chain
chain = prompt | openai_model.with_structured_output(CalendarEvent)

# Run the chain
response = chain.invoke({
    "input_text": "Alice and Bob are going to a science fair on Friday."
})

# Print the formatted result
print(json.dumps(response.dict(), indent=2))

# The response is a CalendarEvent object
response


2025-02-25 10:57:23,245 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{
  "name": "Science Fair",
  "date": "2025-02-28",
  "participants": [
    "Alice",
    "Bob"
  ]
}


CalendarEvent(name='Science Fair', date='2025-02-28', participants=['Alice', 'Bob'])

In [62]:
from langchain_core.output_parsers.json import JsonOutputParser

In [71]:
input_prompt = """As a specialized research assistant, your task is to perform detailed topic analysis
of news item summaries. You will process news items summaries provided as a JSON object according to  
the input specification below. You will extract topics of the news item summaries according to the 
output specification below and return a raw JSON object without any additional formatting or markdown syntax. 

Input Specification:
You will receive an array of JSON objects representing news summaries.
Each headline object contains exactly two fields:
'id': A unique numeric identifier
'summary': The news summmary item

Example input:
[
 {{
    "id": 29,
    "summary": "• Elon Musk's xAI launched Grok 3, a new family of AI models trained using 100,000 Nvidia H100 GPUs at the Colossus Supercluster; benchmarks show it outperforms competitors like GPT-4o and Claude 3.5 Sonnet in areas such as math, science, and coding.

• Grok 3 includes advanced features like reasoning models for step-by-step logical problem-solving and a DeepSearch function that synthesizes internet-sourced information into single answers; it is initially available to X Premium+ subscribers, with advanced features under a paid "SuperGrok" plan.

• Former Tesla AI director Andrej Karpathy and others have confirmed Grok 3's strong performance, with Karpathy noting it is comparable to and slightly better than leading AI models from OpenAI and other competitors."
  }},
{{
    "id": 34,
    "summary": "• Google Gemini has received a memory upgrade that allows it to recall past conversations and summarize previous chats, enhancing its ability to remember user preferences such as interests and professional details. This feature is currently available only to Google One AI Premium subscribers in English, with broader language support expected soon.

• Users retain control over their data with options to delete past conversations, prevent chats from being saved, or set them to auto-delete, although discussions can still be used for AI training unless deleted.

• Similar to OpenAI's ChatGPT persistent memory feature, Gemini's upgrade aims to make chats more practical, though users are advised not to input sensitive information as conversations may be reviewed for quality control."
  }},
 {{
    "id": 47,
    "summary": "• Major tech companies like OpenAI, Google, and Meta are competing to dominate generative AI, though the path to profitability remains uncertain.  

• Chinese start-up DeepSeek has introduced a cost-effective way to build powerful AI, disrupting the market and pressuring established players.

• OpenAI aims to reach 1 billion users, while Meta continues to invest heavily in AI despite market disruptions caused by DeepSeek."
  }},
{{
    "id": 56,
    "summary": "- OpenAI is exploring new measures to protect itself from a potential hostile takeover by Elon Musk.  
- The company is in discussions to empower its non-profit board to maintain control as it transitions into a for-profit business model."
  }},
 {{
    "id": 63,
    "summary": "- The New York Times has approved the use of select AI tools, such as GitHub Copilot, Google Vertex AI, and their in-house summarization tool Echo, to assist with tasks like content summarization, editing, and enhancing product development, while reinforcing the tools as aids rather than replacements for journalistic work.

- Strict guidelines and safeguards have been implemented, including prohibitions on using AI to draft full articles, revise them significantly, or generate images and videos, with a mandatory training video to prevent misuse and protect journalistic integrity.

- Some staff members have expressed concerns about AI potentially compromising creativity and accuracy, leading to skepticism about universal adoption, although the guidelines align with standard industry practices."
  }},
]

Output Specification:
Return a raw JSON object containing 'items', a list of JSON objects, each containing:
'id': Matching the input item's id field.
'extracted_topics': An array of relevant topic strings
Topics should capture:
- The main subject matter
- Key entities (companies, people, products)
- Technical domains, industry sectors, event types

Output Example:
{{items:
 [{{"id": 29, "extracted_topics": ['AI model development', 'xAI Grok capabilities', 'AI advancements']}},
  {{"id": 34, "extracted_topics": ['Google Gemini', 'Interactive AI advancements', 'Digital assistants']}},
  {{"id": 47, "extracted_topics": ['OpenAI', 'Google', 'Meta', 'DeepSeek']}},
  {{"id": 56, "extracted_topics": ['OpenAI', 'non-profit oversight', 'anti-takeover strategies', 'Elon Musk']}},
  {{"id": 63, "extracted_topics": ['New York Times', 'AI in journalism', 'GitHub Copilot', 'Google Vertex AI']}},
 ]
}}

Detailed Guidelines:
The output must strictly adhere to the output specification.
Do not return markdown, return a raw JSON string.
For each input item, output a valid JSON object for each news item in the exact schema provided.
Extract 3-6 relevant topics per news item.
Avoid duplicate or redundant topics.
Use topics which are as specific as possible.
Please analyze the following news items and provide topic classifications according to these specifications:
"""

input_text = \
[
  {
    "id": 0,
    "summary": "4 days left to save up to $325 at TechCrunch Sessions: AI\n* TechCrunch Sessions: AI will take place on June 5 at UC Berkeley's Zellerbach Hall, featuring speakers like Twelve Labs CEO Jae Lee, CapitalG partner Jill Chase, and Khosla Ventures partner Kanu Gulati.  One session will focus on how small companies can stay relevant in the rapidly changing AI space.\n* The event will include main stage talks, breakout sessions, and demos of the latest AI advancements.  It's aimed at AI leaders, VCs, and tech enthusiasts.\n* A discount of up to $325 on select tickets is available until March 2 at 11:59 p.m. PT.\n"
  },
  {
    "id": 1,
    "summary": "57% of enterprise employees input confidential data into AI tools, survey reveals\n* 57% of enterprise employees at companies with 5,000+ staff admitted to inputting confidential company data into publicly available generative AI tools like ChatGPT, according to a TELUS Digital Experience survey of 1,000 US-based employees.\n* 68% of surveyed employees use personal AI accounts for work, indicating a rise in \"shadow AI\" practices that bypass IT and security oversight, increasing data exposure and compliance violation risks.\n* While 29% of respondents confirmed their organizations have AI guidelines, enforcement is weak, with only 24% receiving AI training and 42% reporting no consequences for not following guidelines.  TELUS Digital Fuel iX general manager Bret Kinsella emphasized the need for secure, company-approved AI solutions to address the security risks associated with employees using personal AI accounts for work tasks.\n"
  },
  {
    "id": 2,
    "summary": "A New Machine Learning Approach Answers What-If Questions\n* Causal ML, an emerging machine learning technique, helps managers make better decisions by analyzing potential outcomes of different choices, unlike traditional ML which relies on correlations and may provide flawed insights for decision-making.\n* Causal ML allows for exploring \"what-if\" scenarios, considering various factors and their influence on outcomes, such as determining the optimal R&D budget by considering its impact on revenue alongside other economic variables.\n* While traditional ML remains suitable for predictions like stock prices or customer preferences, Causal ML is valuable for exploring cause-and-effect relationships and informing actions in various business functions like product development, finance, and marketing.\n"
  },
  {
    "id": 3,
    "summary": "A major AI company filed accounts months late and pointed the finger at its Big Four auditor\n* Supermicro filed delayed financial reports, blaming former auditor EY's resignation over concerns about financial reporting governance and senior management integrity.\n* EY dropped Supermicro as a client in October 2024 after raising these concerns, prompting an internal review and the hiring of a new accounting firm, BDO.\n* BDO found no material issues with Supermicro's financials, and the company's stock rebounded after the filings were submitted.\n"
  },
  {
    "id": 4,
    "summary": "AI CAPTCHA Fails Are the Internets New Comedy Show!\n* AI struggles with CAPTCHA challenges, particularly image-based ones, often misidentifying objects or failing to interpret blurred text.  Examples include AI misidentifying a painted bicycle symbol as a real bicycle and failing to transcribe blurred text.\n* While AI can solve some simpler text-based CAPTCHAs, the increasing complexity of CAPTCHAs, including puzzle-based ones, poses a significant challenge even for advanced AI models.\n* Dedicated CAPTCHA-solving tools, unlike AI, are specifically designed to bypass CAPTCHAs and are more effective for this purpose.\n"
  },
  {
    "id": 5,
    "summary": "AI Overview jokes\n* Google's AI Overview feature is producing humorous and inaccurate results for certain queries, including questions about elements ending in \"um\" and the kosher status of tripe.  The inaccuracies seem to stem from the LLMs' difficulty with spelling and counting, as well as a \"lossy-compressed summary of the internet.\"\n* Users report Google is proactively suggesting these flawed queries in the search dropdown, raising questions about whether a human or an algorithm selected them as examples of AI Overview's capabilities.\n*  Several users provided additional examples of inaccurate or nonsensical responses from Google's AI Overview, ranging from incorrect information about surnames of African origin to illogical hyphenation advice and flawed explanations of linguistic history.\n"
  },
  {
    "id": 6,
    "summary": "AI startup Bridgetown Research raises $19 million in latest funding\n* Bridgetown Research, a Seattle-based AI startup, raised $19 million in Series A funding, led by Lightspeed Venture Partners and Accel, with participation from a research university.  The funding round values the company at $250 million.\n* Unlike many AI companies focused on LLMs, Bridgetown Research develops AI agents that collect and analyze proprietary data from experts and customer surveys to provide insights for strategic decision-making.\n* The company plans to use the funding to expand the capabilities of its AI agents and broaden access to sector-specific intelligence through partnerships.\n"
  },
  {
    "id": 7,
    "summary": "AI-Powered Ransomware Attacks\n* AI is being used to enhance ransomware attacks, automating processes like vulnerability analysis, malware deployment, and lateral movement within networks, making them more sophisticated and harder to detect.\n* AI-powered phishing attacks are becoming more targeted and convincing, leveraging publicly available data to create personalized messages and dynamically adjusting content based on recipient behavior.\n* Defending against AI-powered ransomware requires a multi-layered approach including AI-driven security systems, firewalls, updated anti-malware software, intrusion detection systems, end-point detection and response tools, employee training, and robust incident response plans with regular data backups.\n"
  },
  {
    "id": 8,
    "summary": "Akool unleashes enhancements to its AI human 3D avatars connected to LLMs\n* Akool Inc. enhanced its AI-driven 3D human avatars to connect with large language models (LLMs), enabling dynamic conversational experiences.  The avatars can display emotions, movements, and hand gestures, creating a lifelike interaction similar to a video call.\n* Akool offers two avatar types: talking avatars for scripted messages and streaming avatars for real-time conversations, suitable for customer service and guidance.  Users can customize LLMs or integrate with existing models like OpenAI's.\n* Akool CEO Jiajun Lu highlighted the avatars' success in customer service and language education, citing the improved user experience from interacting with a lifelike figure. He also sees potential in government and healthcare.  Low latency and full-body motion capabilities are key differentiators for Akool's technology.\n"
  },
  {
    "id": 9,
    "summary": "Alibaba And DeepSeek Intensify AI Showdown, Challenge OpenAI Market Dominance\n* Chinese AI startup DeepSeek has reopened its core programming interface after a three-week suspension.\n* DeepSeek had previously suspended service due to capacity issues, according to Bloomberg.\n* DeepSeek's reopening intensifies the AI competition with OpenAI and other major players.\n"
  },
  {
    "id": 11,
    "summary": "Amazon's $25 Billion Robotics Push Targets Cost Savings, AI Growth And Temu Competition: Report\n* Amazon has committed up to $25 billion to its retail network, including robotics, aiming for cost savings and AI growth.\n* The investment is partly driven by competition from Temu and the increasing costs of artificial intelligence.\n* The robotics investment has the potential to generate near-term savings for Amazon.\n"
  },
  {
    "id": 12,
    "summary": "Amazons subscription-based Alexa+ looks highly capableand questionable\n* Amazon is launching Alexa+, a more conversational and capable version of its voice assistant, powered by large language models.  It will be free for Prime members and $20/month for non-Prime subscribers.\n* Alexa+ will initially be available on Echo Show 8, 10, 15, and 21 smart displays.  Amazon demonstrated features like personalized recipe recommendations, ticket price monitoring, and seamless integration with other Amazon services like Amazon Music and Fire TV.\n* This upgrade aims to revitalize Amazon's voice assistant business, which has struggled to be profitable, especially in the face of competition from generative AI chatbots.\n"
  },
  {
    "id": 13,
    "summary": "Anthropic's Claude 3.7 Sonnet reportedly cost a few tens of millions of dollars to train, similar to Claude 3.5 and cheaper than GPT-4, which cost over $100M\n* Anthropic's latest AI model, Claude 3.7 Sonnet, reportedly cost \"a few tens of millions of dollars\" to train, using less than 10^26 FLOPs of computing power.\n* This cost is comparable to Claude 3.5 and significantly lower than the reported training costs of OpenAI's GPT-4 (over $100 million) and Google's Gemini Ultra (close to $200 million).\n* While current costs are relatively low, Anthropic CEO Dario Amodei predicts future AI model training will cost billions of dollars.\n"
  },
  {
    "id": 14,
    "summary": "Apple AI tool transcribed the word 'racist' as 'Trump'\n* Apple's speech-to-text tool incorrectly transcribed \"racist\" as \"Trump,\" a problem the company claims is due to difficulty distinguishing words with \"r.\"  A fix is being rolled out.\n* Experts dispute Apple's explanation, citing the distinct phonetic differences and vast training data used for such models.  One expert suggested potential software manipulation, while a former Apple employee called it a \"serious prank.\"\n* This follows another incident where Apple's AI-generated news summaries displayed false information, leading to suspension of the feature.\n"
  },
  {
    "id": 15,
    "summary": "Artificial Intelligence (AI) and the Metaverse Intellectual Property (IP), Standards and Policies Training Course (ONLINE EVENT: March 11, 2025 & ON-DEMAND)\n* A training course titled \"Artificial Intelligence (AI) and the Metaverse: Intellectual Property (IP) and standards and policies\" has been launched by ResearchAndMarkets.com, covering legal and commercial aspects of AI and Metaverse technologies.\n* The course addresses intellectual property issues arising from AI and Metaverse use, AI standards' role in policy development, and the latest UK and EU legislation.  It includes a practical workshop on negotiating IP clauses.\n*  The speakers include Mark Weston, a partner at Hill Dickinson LLP specializing in commercial, IP, and IT law, and Henry Rivero, founder of Riveroconsult with expertise in TV and digital media.\n"
  },
  {
    "id": 16,
    "summary": "Balancing Power With Caution: AIs Impact On Breast Cancer\n* AI is showing promise in breast cancer care, particularly in mammography, potentially increasing detection rates by 20% without increasing false positives.  However, access to AI-enhanced mammography is currently uneven, and research is ongoing to determine if AI can match the effectiveness of dual radiologist readings.\n* While AI can accelerate analysis and personalize treatment, challenges remain, including biased datasets and cost barriers for patients.  Advocacy for insurance coverage and diverse participation in clinical trials are needed to ensure equitable access.\n*  The human element in healthcare must be preserved alongside AI advancements. Technology should complement, not replace, compassionate patient care.\n"
  },
  {
    "id": 17,
    "summary": "Big bang Nvidia Q4 earnings today; here's what you need to watch out for\n* Nvidia's Q4 FY2025 earnings are projected at $38.32 billion in revenue and $21.08 billion in net income, representing significant year-over-year growth driven by increasing demand for AI infrastructure.\n*  Analysts are optimistic about Nvidia's performance, with a majority giving \"buy\" ratings and an average price target of $175.  The company's success is attributed to the rising demand for its data center chips, particularly in the AI sector.\n*  Investors are also keenly awaiting Nvidia's FY2026 guidance, with projected revenue of approximately $42 billion.  The performance and guidance will be key indicators of the overall AI market's trajectory.\n"
  },
  {
    "id": 18,
    "summary": "Billionaire Ray Dalio Says AI Risks 'Totalitarian Control Or Anarchy' As It Could Reshape World In Next 5 Years: Here Are AI-Linked ETFs For Investors To Consider\n* Billionaire Ray Dalio warns that the development of artificial intelligence (AI) could lead to totalitarian control or anarchy in the next five years.\n* Dalio shared his concerns during a recent podcast interview with Tucker Carlson.\n* He emphasized the unpredictable nature of AI's development and its potential societal impact. \n"
  },
  {
    "id": 19,
    "summary": "Bluesky Dubs AI Video of Trump Sucking Elon Musk's Toes 'Non-Consensual Explicit Material'\n* Bluesky removed an AI-generated video depicting Trump engaging in explicit acts with Elon Musk, citing it as \"non-consensual explicit material.\"\n* The video, shared by journalist Marisa Kabas, was originally displayed on hacked TV screens within the Department of Housing and Urban Development (HUD) as a protest.\n* After Kabas appealed the removal, citing the video's political context, Bluesky reinstated it, acknowledging their moderators initially missed the newsworthy context.\n"
  },
  {
    "id": 20,
    "summary": "Cash torrent pouring into Nvidia slows  despite booming Blackwell adoptionMay we all have problems like annual revenue growth dropping from 126 to 114 percentSystems11 hrs|6\n* Nvidia's fiscal year 2025 revenue reached $130 billion (114% growth), slightly lower than the previous year's 126% growth, with Q4 2025 revenue at $39.3 billion, including $11 billion from Blackwell GPUs.  Profits for FY 2025 were $72.9 billion (145% growth).\n*  The company forecasts Q1 2026 revenue around $43 billion, driven by anticipated demand for AI infrastructure, particularly large GPU clusters, and the shift towards widespread inferencing deployments.\n*  While facing geopolitical pressures like export controls and potential tariffs, Nvidia remains optimistic about growth, citing strong demand for Blackwell accelerators, NVLink interconnects, and networking products, and partnerships like the one with Cisco for Spectrum-X.\n"
  }
]


class TopicSpec(BaseModel):
    """TopicSpec class for structured output of story topics"""
    id: int = Field(description="The id of the story")
    extracted_topics: List[str] = Field(
        description="List of topics covered in the story")


class TopicSpecList(BaseModel):
    """List of TopicSpec class for structured output"""
    items: List[TopicSpec] = Field(description="List of TopicSpec")

model = ChatGoogleGenerativeAI(model='models/gemini-2.0-flash', request_timeout=60, verbose=True)
langchain.verbose = True
prompt_template = ChatPromptTemplate.from_messages([
        ("system", input_prompt),
        ("user", "{input_text}")
    ])
input_dict = {"input_text": input_text}

chain = prompt_template | model.with_structured_output(TopicSpecList)
response = chain.invoke(input_dict)
resp = (response.content[8:-3])
    
resp_dict = json.loads(resp.replace("'", '"'))

# Validate with TopicSpecList
try:
    validated_response = TopicSpecList(**resp_dict)
    print(validated_response)
except ValidationError as e:
    print(f"Validation Error: {e.json()}")


ValidationError: 20 validation errors for TopicSpecList
items.0.extracted_topics
  Field required [type=missing, input_value={'id': 0.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
items.1.extracted_topics
  Field required [type=missing, input_value={'id': 1.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
items.2.extracted_topics
  Field required [type=missing, input_value={'id': 2.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
items.3.extracted_topics
  Field required [type=missing, input_value={'id': 3.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
items.4.extracted_topics
  Field required [type=missing, input_value={'id': 4.0}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing
items.5.extracted_topics
  Field required [type=missing, input_value=

In [66]:
print(response)

{'items': [{'id': 0, 'extracted_topics': ['TechCrunch Sessions: AI', 'AI event', 'AI advancements', 'UC Berkeley', 'AI leaders', 'VCs']}, {'id': 1, 'extracted_topics': ['Enterprise AI usage', 'Data security risks', 'Shadow AI', 'TELUS Digital Experience survey', 'ChatGPT', 'AI training']}, {'id': 2, 'extracted_topics': ['Causal ML', 'Machine learning', 'Decision-making', 'What-if scenarios', 'R&D budget optimization', 'Business applications of ML']}, {'id': 3, 'extracted_topics': ['Supermicro', 'Financial reporting', 'EY', 'BDO', 'Auditing', 'Corporate governance']}, {'id': 4, 'extracted_topics': ['AI CAPTCHA failures', 'CAPTCHA challenges', 'Image recognition', 'Text transcription', 'AI limitations', 'CAPTCHA-solving tools']}, {'id': 5, 'extracted_topics': ['Google AI Overview', 'LLM inaccuracies', 'Search engine results', 'Humorous AI responses', 'Algorithm flaws', 'Information retrieval']}, {'id': 6, 'extracted_topics': ['Bridgetown Research', 'AI startup funding', 'AI agents', 'Lig

In [60]:
resp = (response.content[8:-3])
print(resp)


{
  "items": [
    {
      "id": 0,
      "extracted_topics": [
        "TechCrunch Sessions: AI",
        "AI event",
        "UC Berkeley",
        "AI advancements"
      ]
    },
    {
      "id": 1,
      "extracted_topics": [
        "Enterprise AI security risks",
        "Confidential data exposure",
        "Shadow AI",
        "TELUS Digital Experience survey",
        "ChatGPT"
      ]
    },
    {
      "id": 2,
      "extracted_topics": [
        "Causal ML",
        "Machine learning",
        "Decision-making",
        "What-if scenarios"
      ]
    },
    {
      "id": 3,
      "extracted_topics": [
        "Supermicro",
        "EY",
        "Financial reporting",
        "Auditing",
        "BDO"
      ]
    },
    {
      "id": 4,
      "extracted_topics": [
        "AI CAPTCHA failures",
        "Image recognition",
        "AI limitations",
        "CAPTCHA-solving tools"
      ]
    },
    {
      "id": 5,
      "extracted_topics": [
        "Google AI Overview",

In [59]:
import json
from pydantic import ValidationError

# Convert the string to a dictionary
resp = (response.content[8:-3])
    
resp_dict = json.loads(resp.replace("'", '"'))

# Validate with TopicSpecList
try:
    validated_response = TopicSpecList(**resp_dict)
    print(validated_response)
except ValidationError as e:
    print(f"Validation Error: {e.json()}")

items=[TopicSpec(id=0, extracted_topics=['TechCrunch Sessions: AI', 'AI event', 'UC Berkeley', 'AI advancements']), TopicSpec(id=1, extracted_topics=['Enterprise AI security risks', 'Confidential data exposure', 'Shadow AI', 'TELUS Digital Experience survey', 'ChatGPT']), TopicSpec(id=2, extracted_topics=['Causal ML', 'Machine learning', 'Decision-making', 'What-if scenarios']), TopicSpec(id=3, extracted_topics=['Supermicro', 'EY', 'Financial reporting', 'Auditing', 'BDO']), TopicSpec(id=4, extracted_topics=['AI CAPTCHA failures', 'Image recognition', 'AI limitations', 'CAPTCHA-solving tools']), TopicSpec(id=5, extracted_topics=['Google AI Overview', 'LLM inaccuracies', 'Search engine results', 'AI humor']), TopicSpec(id=6, extracted_topics=['Bridgetown Research', 'AI agents', 'Series A funding', 'Lightspeed Venture Partners', 'Accel', 'Strategic decision-making']), TopicSpec(id=7, extracted_topics=['AI-powered ransomware', 'Cybersecurity', 'Phishing attacks', 'Vulnerability analysis',